In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [ ]:
!pip install -q sentence_transformers


In [ ]:
!pip install -q llama-index


In [ ]:
%pip install -q llama-index-llms-huggingface

In [1]:
import logging
import sys
from os import getenv


logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext



/home/ubuntu/projects/llmlegalassistant/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from llama_index.llms.huggingface import HuggingFaceLLM

In [3]:
from llama_index.core import PromptTemplate


system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."



# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate(
    "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."
    "\n\n"
    "### Instruction:\n{query_str}\n\n### Response:"
)

In [8]:
!huggingface-cli login

Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/ubuntu/.cache/huggingface/token
Login successful


In [4]:
from llama_index.core import Settings

In [5]:
import torch
from transformers import BitsAndBytesConfig
from llmlegalassistant.utils import get_api_key

hf_token = get_api_key("HuggingFace")

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_double_quant=True,
)

llm = HuggingFaceLLM(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    context_window=4096,
    max_new_tokens=512,
    query_wrapper_prompt=query_wrapper_prompt,
    system_prompt=system_prompt,
    device_map="auto",
    model_kwargs={"token": hf_token, "quantization_config": quantization_config},
)
Settings.llm = llm

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
model.safetensors.index.json: 100%|██████████| 26.8k/26.8k [00:00<00:00, 120MB/s]
model-00001-of-00002.safetensors: 100%|██████████| 9.98G/9.98G [02:50<00:00, 58.7MB/s]
model-00002-of-00002.safetensors: 100%|██████████| 3.50G/3.50G [00:56<00:00, 61.6MB/s]


INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


ValueError: 
                    Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the
                    quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules
                    in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to
                    `from_pretrained`. Check
                    https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                    for more details.
                    

In [ ]:
documents = SimpleDirectoryReader('/content/drive/MyDrive/data').load_data()

In [ ]:
from llama_index.core.llama_dataset.generator import RagDatasetGenerator

import nest_asyncio

nest_asyncio.apply()

dataset_generator = RagDatasetGenerator.from_documents(
    documents=documents,
    llm=llm,
    num_questions_per_chunk=1,  # set the number of questions per nodes
)

rag_dataset = dataset_generator.generate_dataset_from_nodes()

In [ ]:
#after running this code. see how you can save this data set somewhere.